In [1]:
import os
import os.path as op
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import mne
from library import helpers, config

Study path is set to: /raven/ptmp/fklotzsche/Experiments/vMemEcc
creating dir: /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/05_tfrs_clean
creating dir: /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/05_tfrs_clean/summaries


In [2]:
def get_tfrs_list(subID, part_epo, pwr_style, picks='eeg', save_singletrial_data=False):
    fpath = op.join(config.paths['03_preproc-pooled'], part_epo, "collapsed")
    epos_ = helpers.load_data(f"{subID}-{part_epo}-collapsed", fpath, '-epo')
    
    if (part_epo in ['cue', 'fulllength']):
        # Shift time, so that 0 == Stimulus Onset:
        epos_ = epos_.shift_time(-config.times_dict['cue_dur'])
        
    if pwr_style == 'induced':
        epos_ = epos_.subtract_evoked()

    #  picks = config.chans_CDA_all
    #tfrs_ = get_tfr(epos_, picks=picks, average=False)

    event_dict = helpers.get_event_dict(epos_.event_id)

    sub_tfrs = list()

    for load in ['LoadLow', 'LoadHigh']:
        avgtfrs_load = get_tfr(epos_[event_dict[load]], picks=picks, 
                               average=False)
        avgtfrs_load.comment = load
        if save_singletrial_data:
            save_tfr(subID, avgtfrs_load, load, pwr_style, part_epo)
        sub_tfrs.append(avgtfrs_load)
        # Save averaged version:   # <<<< this saving per condition is probably redundant in the latest version of the code
        # save_tfr(subID, avgtfrs_load.average(), load, pwr_style, part_epo, averaged=True)
        for ecc in ['EccS', 'EccM', 'EccL']:
            if load == 'LoadLow':  # we don't want to do this twice
                avgtfrs_ecc = get_tfr(epos_[event_dict[ecc]], picks=picks, 
                                      average=False)
                avgtfrs_ecc.comment = ecc
                if save_singletrial_data:
                    save_tfr(subID, avgtfrs_ecc, ecc, pwr_style, part_epo)
                sub_tfrs.append(avgtfrs_ecc)
                # save_tfr(subID, avgtfrs_ecc.average(), ecc, pwr_style, part_epo, averaged=True)
            # Interaction:
            avgtfrs_interac = get_tfr(epos_[event_dict[load]][event_dict[ecc]],
                                      picks=picks, average=False)
            avgtfrs_interac.comment = load+ecc
            if save_singletrial_data:
                save_tfr(subID, avgtfrs_interac, load+ecc, pwr_style, part_epo)
            sub_tfrs.append(avgtfrs_interac)
            # save_tfr(subID, avgtfrs_interac.average(), load+ecc, pwr_style, part_epo, averaged=True)
    avgtfrs_all = get_tfr(epos_, picks=picks, average=False)
    avgtfrs_all.comment = 'all'
    if save_singletrial_data:
            save_tfr(subID, avgtfrs_all, 'all', pwr_style, part_epo)
    sub_tfrs.append(avgtfrs_all)
    # save_tfr(subID, avgtfrs_all.average(), 'all', pwr_style, part_epo, averaged=True)

    fpath = op.join(config.paths['05_tfrs'], pwr_style, 'tfr_lists', part_epo)
    helpers.chkmk_dir(fpath)
    if save_singletrial_data:
        fname = op.join(fpath, subID + '-collapsed-singletrialTFRs-tfr.h5')
        mne.time_frequency.write_tfrs(fname, sub_tfrs, overwrite=True)
    fname = op.join(fpath, subID + '-collapsed-avgTFRs-tfr.h5')
    mne.time_frequency.write_tfrs(fname, [t.average() for t in sub_tfrs], overwrite=True)
    return(sub_tfrs)


def save_tfr(subID, sub_tfrs, condition, pwr_style='induced', part_epo='fulllength', averaged=False):
    fpath = op.join(config.paths['05_tfrs'], pwr_style, 'tfr_lists', part_epo, condition)
    helpers.chkmk_dir(fpath)
    if averaged:
        fname = op.join(fpath, subID + '-collapsed-avgTFRs-tfr.h5')
    else:
        fname = op.join(fpath, subID + '-collapsed-singletrialTFRs-tfr.h5')  
    
    mne.time_frequency.write_tfrs(fname, sub_tfrs, overwrite=True)


def get_tfr(epos, picks='all', average=True, freqs=None):
    if freqs is None:
        freqs = np.concatenate([np.arange(6, 26, 1)])  # , np.arange(16,30,2)])
    n_cycles = freqs / 2.  # different number of cycle per frequency
    power = mne.time_frequency.tfr_morlet(epos,
                                          picks=picks,
                                          freqs=freqs,
                                          n_cycles=n_cycles,
                                          use_fft=True,
                                          return_itc=False,
                                          average=average,
                                          decim=1,
                                          n_jobs=-2)
    return power

In [5]:
# structuring data:
sub_list = np.setdiff1d(np.arange(1, 28), config.ids_missing_subjects +
                        config.ids_excluded_subjects)               
sub_list_str = ['VME_S%02d' % sub for sub in sub_list]

# when running on the cluster we want parallelization along the subject dimension
if not helpers.is_interactive(): 
    helpers.print_msg('Running Job Nr. ' + sys.argv[1])
    job_nr = int(float(sys.argv[1]))
    sub_list_str = [sub_list_str[job_nr]]   

part_epo = 'fulllength'
pwr_style = 'induced'  

for subID in sub_list_str:
    print(f"Running {subID} ...")
    _ = get_tfrs_list(subID, part_epo, pwr_style, picks="eeg", save_singletrial_data=False)

Running VME_S01 ...
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/pooled/fulllength/collapsed/VME_S01-fulllength-collapsed-epo.fif ...
    Found the data of interest:
        t =    -600.00 ...    3300.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
659 matching events found
No baseline correction applied
0 projection items activated
Subtracting Evoked from Epochs
    The following channels are not included in the subtraction: IO1, IO2, LO2, LO1, VEOG, ECG, HEOG
[done]


[Parallel(n_jobs=143)]: Using backend LokyBackend with 143 concurrent workers.
[Parallel(n_jobs=143)]: Done   9 out of  60 | elapsed:   12.2s remaining:  1.2min
[Parallel(n_jobs=143)]: Done  22 out of  60 | elapsed:   14.3s remaining:   24.6s
[Parallel(n_jobs=143)]: Done  35 out of  60 | elapsed:   16.2s remaining:   11.6s
[Parallel(n_jobs=143)]: Done  48 out of  60 | elapsed:   18.2s remaining:    4.6s
[Parallel(n_jobs=143)]: Done  60 out of  60 | elapsed:   20.0s finished


Not setting metadata
creating dir: /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/05_tfrs_clean/induced/tfr_lists/fulllength/LoadLow


[Parallel(n_jobs=143)]: Using backend LokyBackend with 143 concurrent workers.
[Parallel(n_jobs=143)]: Done   9 out of  60 | elapsed:    3.3s remaining:   18.7s
[Parallel(n_jobs=143)]: Done  22 out of  60 | elapsed:    4.7s remaining:    8.1s
[Parallel(n_jobs=143)]: Done  35 out of  60 | elapsed:    5.9s remaining:    4.2s
[Parallel(n_jobs=143)]: Done  48 out of  60 | elapsed:    7.2s remaining:    1.8s
[Parallel(n_jobs=143)]: Done  60 out of  60 | elapsed:    8.3s finished


Not setting metadata
creating dir: /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/05_tfrs_clean/induced/tfr_lists/fulllength/EccS


...

Not setting metadata


[Parallel(n_jobs=143)]: Using backend LokyBackend with 143 concurrent workers.
[Parallel(n_jobs=143)]: Done   9 out of  60 | elapsed:    0.6s remaining:    3.4s
[Parallel(n_jobs=143)]: Done  22 out of  60 | elapsed:    1.1s remaining:    2.0s
[Parallel(n_jobs=143)]: Done  35 out of  60 | elapsed:    1.7s remaining:    1.2s
[Parallel(n_jobs=143)]: Done  48 out of  60 | elapsed:    2.2s remaining:    0.6s
[Parallel(n_jobs=143)]: Done  60 out of  60 | elapsed:    2.8s finished


Not setting metadata


[Parallel(n_jobs=143)]: Using backend LokyBackend with 143 concurrent workers.
[Parallel(n_jobs=143)]: Done   9 out of  60 | elapsed:    4.6s remaining:   26.0s
[Parallel(n_jobs=143)]: Done  22 out of  60 | elapsed:    8.5s remaining:   14.6s
[Parallel(n_jobs=143)]: Done  35 out of  60 | elapsed:   12.4s remaining:    8.8s
[Parallel(n_jobs=143)]: Done  48 out of  60 | elapsed:   16.7s remaining:    4.2s
[Parallel(n_jobs=143)]: Done  60 out of  60 | elapsed:   21.0s finished


Not setting metadata
